In [1]:
import matlab.engine
import numpy as np
import scipy.optimize

# Trial and pd parameters

In [47]:
# trial and adherence information
#drug = "ATV/r"
drug = "DTG"

days = np.double(40) #days to run trial for
increment = 0.01 # days between each timepoint returned
prob_adh = 0.5 if drug != 'TEST' else 0. # probability of taking each pill (independent)
adh_pat = 0 # 0 = random prob for each dose, 1 = random prob each day after missed dose
adh_shuff = np.double(0.) # standard deviation in the time of taking a pill vs scheduled (days)
trial_type = 1 # 1 = suppression trial, 0=maintenance trial
burnin = 7*5 # days before interval sampling begins in maintenance trials
sampling_int = 28 # days between each sampling time in maintenance trials
threshold = 200 # threshold viral load above which failure is declared
mut_frac = 0.2 # threshold fraction of population of mutant above which declared failure via resistance

## Use existing matlab scripts to calculate concentration(t)

In [48]:
eng = matlab.engine.start_matlab()
eng.cd(r'~/develop/withinhostHIV/MatlabCode/', nargout=0)
eng.eval("addpath('Parameters','Utilities')", nargout=0)

In [49]:
eng.workspace["drug"] = drug
trial, pd = eng.trial_setup(eng.eval("drug"), days, increment, prob_adh, adh_pat, adh_shuff, trial_type, burnin, sampling_int, threshold, mut_frac, nargout=2)
eng.workspace["trial"] = trial
eng.workspace["pd"] = pd

In [50]:
dose_t, _ = eng.adh_trajectory(eng.eval("pd.num_doses"), trial, nargout=2)

In [51]:
c_vec,inhib_vec = eng.drug_trajectory(pd, trial, dose_t, nargout=2)

## Use existing matlab scripts to calculate mutation matrix Q

In [52]:
smin=0.05; #highest mutant fitness
smax=0.9; #lowest mutant fitness
smiss=0.05; #fitness of strains with missing cost
rfmiss=1; #fold change in resistance for strains with it missing
mfmiss=0; #fractional change in slope for strains with it missing
back_mutation_on = drug != 'TEST' #include(1) or exclude(0) back mutation
direct_multi_hit = drug != 'TEST' #include(1) or exclude(0) direct multi-hit mutations from WT
direct_multi_multi = drug != 'TEST' #include(1) or exclude(0) direct multi-hit mutations from one res strain to another

In [53]:
mparams,Q = eng.getMutInfo(drug,smin,smax,smiss,rfmiss,mfmiss,back_mutation_on,direct_multi_hit,direct_multi_multi,nargout=2);
eng.workspace["Q"] = Q
Q = eng.eval("Q{:,:};") # get a matrix instead of a matlab table

## Viral dynamics parameters

In [54]:
R00 = 10.0 # average number of new virions produced by infection with a single virion === average number of new infected cells produced by 1 infected cell
fbp = 0.55 # fraction of blood that is plasma
Vpl = 3*10e3 # volume of plasma in mL
ftcell_pl = 0.02 # fraction of T cells that are circulating in blood (vs in lymph system)
hl_lr = 44*30.5 # half life of decay of latent reservoir, days
A = 100 # total reactivation of latent cells per day
flr = 1e-6# fraction of CD4 T cells that are latently infected at equilibrium

scale_cd4_body = (Vpl*10**3)/(fbp*ftcell_pl) # factor to go from T cell per ul blood to whole body

fa = 0.01 # fraction of infected CD4 cells that support productive vs abortive infection
dx = 0.05 # death rate of uninfected cells (per day) and by assumption dx == d, rate of death without viral cytolytic effects
L = scale_cd4_body*1000*dx # uninfected cells produced per day (/ul)
a = A/(flr*L/dx) # rate of exit of latent cells (per day)
dz = np.log(2)/(hl_lr)-a # death rate of latently infected cells (per day)

dy = 1 # TOTAL death rate of infected cells (per day) (=death due to burst (k) + death without viral cytolytic effects)
k = dy-dx # rate of death+emission of a burst of virions (per day)
N = 2.38e5
assert(N > R00)

# probability of a single virion establishing infection, solved implicitly
p_est_solution = scipy.optimize.least_squares(lambda p_est: R00*(1-(1-p_est)**N) - N*p_est, 0.1)
assert(p_est_solution.success)
p_est = p_est_solution.x

dv = 25 # TOTAL death rate of virions (per day)
beta = R00 * dy * dv * dx / (k * N * fa * L) # infectivity rate (per day*infectious-target cell pair)
c = dv-beta*L/dx # clearance rate of virions (per day);
assert(c > 0)
g = flr*dy/dx*(a+dz)/(fa*(1-1/R00)) # fraction of new infections that become latent

In [55]:
# Access matlab parameters
eng.workspace["mparams"] = mparams
IC50 = eng.eval('pd.IC50')
m = eng.eval('pd.m')
cost = eng.eval('mparams.cost')
rf = eng.eval('mparams.rf')
mf = eng.eval('mparams.mf')
msb = np.asarray(eng.eval("mparams.msb"))
t_vec = eng.eval('(0:trial.increment:trial.days);')


## Check beta(t) (from matlab) against beta(t) (from python)

In [56]:
_beta_t, _beta_u_t = eng.calculate_beta_of_t(t_vec, beta, c_vec, IC50, m, cost, rf, mf, eng.eval('height(mparams)'), nargout=2)

In [57]:
from numba import jit

@jit(nopython=True)
def calculate_beta_t(beta_0, concentration, IC50, m, cost, rf, mf):
    # B/[cost * (1 + concentration)/(IC50*rf)^(m*(1+mf))]
    B = beta_0
    denominator = cost * (1 + (concentration/(IC50*rf))**(m*(1+mf)))
    return B/denominator


# make sure we didn't mess up our function definition
assert(np.allclose(calculate_beta_t(beta, np.asarray(c_vec), IC50, m, np.asarray(cost).T, np.asarray(rf).T, np.asarray(mf).T), np.asarray(_beta_u_t)))
assert(np.allclose(calculate_beta_t(beta, np.asarray(c_vec), IC50, m, 1, 1, 1), np.asarray(_beta_t)))

beta_u_t = calculate_beta_t(beta, np.asarray(c_vec), IC50, m, np.asarray(cost).T, np.asarray(rf).T, np.asarray(mf).T)
beta_t   = calculate_beta_t(beta, np.asarray(c_vec), IC50, m, 1, 1, 1)

beta_t = np.concatenate([beta_t, beta_u_t], axis=1)
t_vec = np.asarray(t_vec).squeeze()
dose_t = np.asarray(dose_t).T.squeeze()

In [58]:
beta_t

array([[4.05425328e-14, 2.59888031e-14, 2.14510756e-14, ...,
        4.46012462e-15, 2.83514215e-14, 3.85154062e-14],
       [4.05425328e-14, 2.59888031e-14, 2.14510756e-14, ...,
        4.46012462e-15, 2.83514215e-14, 3.85154062e-14],
       [4.05425328e-14, 2.59888031e-14, 2.14510756e-14, ...,
        4.46012462e-15, 2.83514215e-14, 3.85154062e-14],
       ...,
       [1.10966475e-17, 1.42420223e-16, 2.63631929e-16, ...,
        6.46310468e-16, 8.75549967e-15, 8.00188382e-15],
       [1.14175674e-17, 1.44454176e-16, 2.67333155e-16, ...,
        6.54230752e-16, 8.84202431e-15, 8.09265462e-15],
       [1.17477658e-17, 1.46517011e-16, 2.71085680e-16, ...,
        6.62231243e-16, 8.92901441e-15, 8.18417892e-15]])

In [59]:
sampled_concentration = np.asarray(c_vec)

increment = eng.eval('trial.increment')

# closed around parameters
# pulls the nearest beta pre-calculated to the left of the current time
@jit(nopython=True)
def discrete_beta_of_t(t):
    # last time where beta was precalculated *before* t
    # specifically want *before* t so that we don't look ahead while integrating past a discontinuity
    t_discrete = int(np.floor(t/increment))
    return beta_t[t_discrete]

Q = np.asarray(Q)
@jit(nopython=True)
def force_of_infection(t):
    beta = discrete_beta_of_t(t)
    # calculate force of infection and flatten in row major order, which will make the matrix
    # read correctly in our 1-dimensional vector of rates

    # **NOTE* we moved FA to the stoichiometry matrix, otherwise it would be a factor below!
    return (np.expand_dims(beta,1) * Q).flatten()

TODO: triple check to make sure this is the right orientation of beta_i relative to Q

In [60]:
Q

array([[1.00000000e+00, 5.68817141e-05, 2.30298641e-05, 6.70620209e-06,
        1.15304424e-05, 5.49009660e-07, 8.78426471e-07, 3.03706834e-05,
        5.86368000e-05, 2.50604409e-05, 3.81460270e-10, 2.66784122e-11,
        2.65544522e-10, 7.61102717e-10, 1.78083969e-09, 6.33032428e-12],
       [1.15303475e-05, 1.00000000e+00, 2.65542335e-10, 7.73248403e-11,
        1.32950008e-10, 6.33027214e-12, 1.01285624e-11, 3.50184532e-10,
        6.76102678e-10, 2.88955592e-10, 6.70620209e-06, 3.07611363e-16,
        3.06182061e-15, 8.77577878e-15, 2.05337004e-14, 7.29908385e-17],
       [5.31463568e-06, 3.02305587e-10, 1.00000000e+00, 3.56410209e-11,
        6.12801007e-11, 2.91778633e-12, 4.66851666e-12, 1.61409117e-10,
        3.11633229e-10, 1.33187114e-10, 2.02732236e-15, 1.41786041e-16,
        1.15304424e-05, 4.04498365e-15, 9.46451413e-15, 3.36433672e-17],
       [5.49016545e-07, 3.12290021e-11, 1.26437764e-11, 1.00000000e+00,
        6.33040366e-12, 3.01415386e-13, 4.82270666e-13, 1.667

# Define model

In [61]:
import reactionmodel.model as reactionmodel
num_strains = beta_t.shape[1]
num_strains

16

In [62]:
ys = [reactionmodel.Species(f'infected cells strain {i}', f'y{i}') for i in range(num_strains)]
zs = [reactionmodel.Species(f'latent cells strain {i}', f'z{i}') for i in range(num_strains)]
vs = [reactionmodel.Species(f'viruses strain {i}', f'v{i}') for i in range(num_strains)]
x = reactionmodel.Species("target cells", "x")
all_species = [x] + ys + vs + zs

In [63]:
target_cell_birth = reactionmodel.Reaction([], [x], description="target cell birth", k=L)
target_cell_death = reactionmodel.Reaction([x], [], description="target cell death", k=dx)
infected_death = [reactionmodel.Reaction([ys[i]], [], description=f"death of infected cell strain {i}", k=(dy-k)) for i in range(num_strains)]
infected_cell_burst = [reactionmodel.Reaction([ys[i]], [(vs[i], N)], description=f"burst of infected cell strain {i}", k=k) for i in range(num_strains)]
into_latency = [reactionmodel.Reaction([ys[i]], [zs[i]], description=f"strain {i} --> latency", k=g) for i in range(num_strains)]
out_of_latency = [reactionmodel.Reaction([zs[i]], [ys[i]], description=f"latent --> strain {i}", k=a) for i in range(num_strains)]
death_of_latent = [reactionmodel.Reaction([zs[i]], [], description=f"latent {i} death", k=dz) for i in range(num_strains)]
viral_clearance = [reactionmodel.Reaction([vs[i]], [], description=f"clearance of strain {i} virion", k=c) for i in range(num_strains)]

infections = []
for i in range(num_strains):
    for j in range(num_strains):
        infections.append(reactionmodel.Reaction([x, vs[i]], [(ys[j], fa)], description=f"infection of x by {i}->{j}"))
infection_family = reactionmodel.ReactionRateFamily(infections, k=force_of_infection)

all_reactions = [target_cell_birth, target_cell_death] + infected_death + infected_cell_burst + into_latency + out_of_latency + death_of_latent + viral_clearance + [infection_family]


In [64]:
for r in all_reactions:
    if isinstance(r, list):
        print(r)

In [65]:
from numba.core.registry import CPUDispatcher
isinstance(all_reactions[-1].k, CPUDispatcher)

True

In [66]:
model = reactionmodel.Model(all_species, all_reactions)

In [67]:
#for reaction in model.all_reactions:
#    print(reaction)

Reaction(description=target cell birth, reactants=(), products=(Species(name='target cells', description='x'),), kinetic_order=(), k=136363636363.63635)
Reaction(description=target cell death, reactants=(Species(name='target cells', description='x'),), products=(), kinetic_order=(Species(name='target cells', description='x'),), k=0.05)
Reaction(description=death of infected cell strain 0, reactants=(Species(name='infected cells strain 0', description='y0'),), products=(), kinetic_order=(Species(name='infected cells strain 0', description='y0'),), k=0.050000000000000044)
Reaction(description=death of infected cell strain 1, reactants=(Species(name='infected cells strain 1', description='y1'),), products=(), kinetic_order=(Species(name='infected cells strain 1', description='y1'),), k=0.050000000000000044)
Reaction(description=death of infected cell strain 2, reactants=(Species(name='infected cells strain 2', description='y2'),), products=(), kinetic_order=(Species(name='infected cells s

# Run forward simulation

In [68]:
from hybrid.hybrid import HybridSimulator, FixedThresholdPartitioner, NThresholdPartitioner

In [69]:
ys

[Species(name='infected cells strain 0', description='y0'),
 Species(name='infected cells strain 1', description='y1'),
 Species(name='infected cells strain 2', description='y2'),
 Species(name='infected cells strain 3', description='y3'),
 Species(name='infected cells strain 4', description='y4'),
 Species(name='infected cells strain 5', description='y5'),
 Species(name='infected cells strain 6', description='y6'),
 Species(name='infected cells strain 7', description='y7'),
 Species(name='infected cells strain 8', description='y8'),
 Species(name='infected cells strain 9', description='y9'),
 Species(name='infected cells strain 10', description='y10'),
 Species(name='infected cells strain 11', description='y11'),
 Species(name='infected cells strain 12', description='y12'),
 Species(name='infected cells strain 13', description='y13'),
 Species(name='infected cells strain 14', description='y14'),
 Species(name='infected cells strain 15', description='y15')]

In [70]:
extra_discontinuities = list(np.linspace(0.0,40.0,400))
extra_discontinuities = []

y0 = np.zeros(len(all_species))
y0[model.species_index[x]] = L/dx/R00

expanded_msb = np.squeeze(np.vstack([np.array([1]), msb]))
y0[model.species_index[ys[0]] : model.species_index[ys[-1]] + 1] = fa*(1-1/R00)*L/dy * expanded_msb
y0[model.species_index[zs[0]] : model.species_index[zs[-1]] + 1] = y0[model.species_index[ys[0]] : model.species_index[ys[-1]] + 1] * g/(a+dz)
y0[model.species_index[vs[0]] : model.species_index[vs[-1]] + 1] = y0[model.species_index[ys[0]] : model.species_index[ys[-1]] + 1] * k * N / (c+beta*L/dx) #y*k*N/(c+B*L/dx)

In [71]:
y0[model.species_index[zs[0]] : model.species_index[zs[-1]] + 1]

array([2.72727273e+06, 4.31595846e+02, 1.33208269e+02, 2.40282115e+01,
       6.51760964e+01, 4.31681625e+00, 1.27953071e+01, 2.75099727e+02,
       1.93429285e+03, 6.37079263e+02, 3.97850012e-03, 1.11518310e-02,
       4.07982819e-03, 2.71571852e-02, 2.32877798e-01, 1.36629689e-03])

In [72]:
y0[model.species_index[zs[3]]] / (L/dx) * 1e6

8.81034420933936e-06

In [73]:
%load_ext jupyterflame

The jupyterflame extension is already loaded. To reload it, use:
  %reload_ext jupyterflame


In [74]:
simulator.simulation_options

HybridSimulationOptions(approximate_rtot=False, contrived_no_reaction_rate=None, deqs=True, round_randomly=True, halt_on_partition_change=False, partition_fraction_for_halt=0.9)

In [75]:
# the adherence pattern and hence the R0s is being determined in matlab before any of this code runs,
# so don't be scared if trajectories look almost identical with different seeds in python

simulator = HybridSimulator(
    model.get_k(jit=True),
    model.stoichiometry(),
    model.kinetic_order(),
    NThresholdPartitioner(100),
    discontinuities=np.sort(np.array(list(dose_t)+extra_discontinuities)),
    jit=True
)

In [79]:
simulator = HybridSimulator(
    model.get_k(jit=True),
    model.stoichiometry(),
    model.kinetic_order(),
    FixedThresholdPartitioner(100),
    discontinuities=np.sort(np.array(list(dose_t)+extra_discontinuities)),
    jit=True
)

In [81]:
rng = np.random.default_rng(0)
result = simulator.simulate([0, 0.1], y0, rng)

print(result.status_counter)

Counter({<HybridStepStatus.stochastic_event: 1>: 75, <HybridStepStatus.t_end: 0>: 1})


In [ ]:
np.sort(np.array(list(dose_t)+extra_discontinuities)),

## R_tot approximation

In [ ]:
# the adherence pattern and hence the R0s is being determined in matlab before any of this code runs,
# so don't be scared if trajectories look almost identical with different seeds in python

simulator = HybridSimulator(
    model.get_k(jit=True),
    model.stoichiometry(),
    model.kinetic_order(),
    FixedThresholdPartitioner(100).partition_function,
    discontinuities=np.sort(np.array(list(dose_t)+extra_discontinuities)),
    jit=True,
    approximate_rtot=True,
    contrived_no_reaction_rate=1.,
    partition_fraction_for_halt=0.8
)

rng = np.random.default_rng(0)
result = simulator.simulate([0, 40.0], y0, rng)

print(result.status_counter)

In [ ]:
result

In [ ]:
scale_cd4_body=(Vpl*10**3)/(fbp*ftcell_pl)
#scale_cd4_virus=kdv*2*1e3/fbp

In [ ]:
mnames = list(np.asarray(eng.eval('mparams.Properties.RowNames')))

import matplotlib.pyplot as plt
plt.figure()
plt.plot(result.t_history, result.y_history[0].T/scale_cd4_body, label='x')
plt.yscale('linear')
plt.ylabel('uninfected CD4 cells (cells/uL)')
plt.xlabel('t')
plt.legend()
plt.ylim([0, 1025])

plt.figure()
plt.plot(result.t_history, result.y_history[1:1+len(ys)].T, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('total infected cells')
plt.xlabel('t')
plt.legend()
plt.ylim([1, 1e10])

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(vs[0]):model.species.index(vs[-1])+1].T/Vpl/1000, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('virions (copies/mL plasma)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-7, 1e8])



plt.figure()
plt.plot(t_vec, beta_t*R00/beta, label=['wildtype']+mnames)
plt.legend()
plt.ylabel('R_u')
plt.xlabel('t')
plt.yscale('linear')

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(zs[0]):model.species.index(zs[-1])+1].T*(10**6)/(L/dx), label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('LR size(per 10^6 baseline CD4)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-7, 10e2])

## N specimen partitioning

In [ ]:
# the adherence pattern and hence the R0s is being determined in matlab before any of this code runs,
# so don't be scared if trajectories look almost identical with different seeds in python

simulator = HybridSimulator(
    model.get_k(jit=True),
    model.stoichiometry(),
    model.kinetic_order(),
    NThresholdPartitioner(100).partition_function,
    discontinuities=np.sort(np.array(list(dose_t)+extra_discontinuities)),
    jit=True
)

rng = np.random.default_rng(2)
result = simulator.simulate([0, 40.0], y0, rng)

print(result.status_counter)

In [ ]:
mnames = list(np.asarray(eng.eval('mparams.Properties.RowNames')))

plt.figure()
plt.plot(result.t_history, result.y_history[0].T/scale_cd4_body, label='x')
plt.yscale('linear')
plt.ylabel('uninfected CD4 cells (cells/uL)')
plt.xlabel('t')
plt.legend()
plt.ylim([0, 1025])

plt.figure()
plt.plot(result.t_history, result.y_history[1:1+len(ys)].T/scale_cd4_body, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('infected cells (cells/uL)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-9, 1e3])

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(vs[0]):model.species.index(vs[-1])+1].T/Vpl/1000, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('virions (copies/mL plasma)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-3, 1e7])

plt.figure()
plt.plot(t_vec, beta_t*R00/beta, label=['wildtype']+mnames)
plt.legend()
plt.ylabel('R_u')
plt.xlabel('t')
plt.yscale('linear')

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(zs[0]):model.species.index(zs[-1])+1].T*(10**6)/(L/dx), label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('LR size(per 10^6 baseline CD4)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-7, 10e2])

# Other

In [ ]:
def dydt_factory(calculate_propensities):
    def dydt(t, y_expanded, k, deterministic_mask, stochastic_mask, hitting_point):
        y = y_expanded[:-1]
        k = k(t)
        propensities = calculate_propensities(y, k)
        deterministic_propensities = propensities * deterministic_mask
    
        deterministic_propensities
        dydt = np.zeros(len(y_expanded))
    
        ## reaction rates used multiple times 
        _infections = deterministic_propensities[r.index(viral_clearance[-1])+1:len(r)].reshape(n, n)
        new_infections_by_strain = _infections.sum(axis=0)
        loss_of_virions_by_strain = _infections.sum(axis=1)
    
        ic_death = deterministic_propensities[r.index(infected_death[0]):r.index(infected_death[0])+n]
        ic_burst = deterministic_propensities[r.index(infected_cell_burst[0]):r.index(infected_cell_burst[0])+n]
        to_latent = deterministic_propensities[r.index(into_latency[0]):r.index(into_latency[0])+n]
        from_latent = deterministic_propensities[r.index(out_of_latency[0]):r.index(out_of_latency[0])+n]
    
        ## Infected cells
        dydt[1:1+n] = (
            fa * new_infections_by_strain
            + from_latent
            - ic_death
            - ic_burst
            - to_latent
        )
    
        ## Virions
        dydt[1+n:1+2*n] = (
            burst_size * ic_burst
            - loss_of_virions_by_strain
            - deterministic_propensities[r.index(viral_clearance[0]):r.index(viral_clearance[0])+n]
        )
    
        ## Latent cells
        dydt[species.index(zs[0]):species.index(zs[0])+n] = (
            to_latent
            - from_latent
            - deterministic_propensities[r.index(death_of_latent[0]):r.index(death_of_latent[0])+n]
        )
    
        ## Target cells
        dydt[species.index(x)] = deterministic_propensities[r.index(target_cell_birth)] - deterministic_propensities[r.index(target_cell_death)] - np.sum(loss_of_virions_by_strain)
        dydt[-1] = np.sum(propensities * stochastic_mask)
        return dydt
    return dydt

In [ ]:
#rates = N @ deterministic_propensities

#fa = 0.01
species = tuple(model.species.copy())
r = tuple(model.all_reactions.copy())
n = len(ys)
burst_size = N

num_pathways = len(r)
clearance_start = r.index(viral_clearance[0])
clearance_end = r.index(viral_clearance[-1])
ic_death_start = r.index(infected_death[0])
ic_burst_start = r.index(infected_cell_burst[0])
to_latent_start = r.index(into_latency[0])
from_latent_start = r.index(out_of_latency[0])
latent_death_start = r.index(death_of_latent[0])
target_birth = r.index(target_cell_birth)
target_death = r.index(target_cell_death)

latent_start = species.index(zs[0])
ic_start = species.index(ys[0])
virus_start = species.index(vs[0])
x_start = species.index(x)

def jit_dydt_factory(_):
    @jit(nopython=True)
    def dydt(t, y_expanded, k, deterministic_mask, stochastic_mask, calculate_propensities, hitting_point):
        y = y_expanded[:-1]
        propensities = calculate_propensities(y, k, t)
        deterministic_propensities = propensities * deterministic_mask
    
        deterministic_propensities
        dydt = np.zeros(len(y_expanded))
    
        ## reaction rates used multiple times 
        _infections = deterministic_propensities[clearance_end+1:clearance_end+1+n**2].reshape(n, n)
        new_infections_by_strain = _infections.sum(axis=0)
        loss_of_virions_by_strain = _infections.sum(axis=1)
    
        ic_death = deterministic_propensities[ic_death_start:ic_death_start+n]
        ic_burst = deterministic_propensities[ic_burst_start:ic_burst_start+n]
        to_latent = deterministic_propensities[to_latent_start:to_latent_start+n]
        from_latent = deterministic_propensities[from_latent_start:from_latent_start+n]
    
        ## Infected cells
        dydt[ic_start:ic_start+n] = (
            fa * new_infections_by_strain
            + from_latent
            - ic_death
            - ic_burst
            - to_latent
        )
    
        ## Virions
        dydt[virus_start:virus_start+n] = (
            burst_size * ic_burst
            - loss_of_virions_by_strain
            - deterministic_propensities[clearance_start:clearance_start+n]
        )
    
        ## Latent cells
        dydt[latent_start:latent_start+n] = (
            to_latent
            - from_latent
            - deterministic_propensities[latent_death_start:latent_death_start+n]
        )
    
        ## Target cells
        dydt[x_start] = (
            deterministic_propensities[target_birth]
            - deterministic_propensities[target_death]
            - np.sum(loss_of_virions_by_strain)
        )
        
        dydt[-1] = np.sum(propensities * stochastic_mask)
        return dydt
    def wrapper(t, y_expanded, k, partition, calculate_propensities, hitting_point):
        return dydt(t, y_expanded, k, partition.deterministic, partition.deterministic, calculate_propensities, hitting_point)
    return wrapper

# Tau leaping

In [ ]:
from hybrid.tau import TauLeapSimulator

simulator = TauLeapSimulator(
    model.get_k(jit=True),
    model.stoichiometry(),
    model.kinetic_order(),
    #discontinuities=np.sort(np.array(list(dose_t)+extra_discontinuities)),
    jit=True,
    epsilon=0.05
)

In [ ]:
model.species

In [ ]:
y0

In [ ]:
dose_t

In [ ]:
result = simulator.simulate([0, 4.2], np.round(y0), np.random.default_rng())

result.status_counter

In [ ]:
mnames = list(np.asarray(eng.eval('mparams.Properties.RowNames')))

import matplotlib.pyplot as plt
plt.figure()
plt.plot(result.t_history, result.y_history[0].T/scale_cd4_body, label='x')
plt.yscale('linear')
plt.ylabel('uninfected CD4 cells (cells/uL)')
plt.xlabel('t')
plt.legend()
plt.ylim([0, 1025])

plt.figure()
plt.plot(result.t_history, result.y_history[1:1+len(ys)].T/scale_cd4_body, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('infected cells (cells/uL)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-9, 1e3])

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(vs[0]):model.species.index(vs[-1])+1].T/Vpl/1000, label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('virions (copies/mL plasma)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-3, 1e7])

plt.figure()
plt.plot(t_vec, beta_t*R00/beta, label=['wildtype']+mnames)
plt.legend()
plt.ylabel('R_u')
plt.xlabel('t')
plt.yscale('linear')

plt.figure()
plt.plot(result.t_history, result.y_history[model.species.index(zs[0]):model.species.index(zs[-1])+1].T*(10**6)/(L/dx), label=['wildtype']+mnames)
plt.yscale('log')
plt.ylabel('LR size(per 10^6 baseline CD4)')
plt.xlabel('t')
plt.legend()
plt.ylim([1e-7, 10e2])